## Coding Applications with Gradient Descent

In [1]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, kstest, anderson, shapiro
# for comparison only
from sklearn.linear_model import LinearRegression, Ridge

In [2]:
# import data
data = pd.read_csv('https://raw.githubusercontent.com/dvasiliu/AML/main/Data%20Sets/Advertising.csv?raw=true')

In [3]:
# here the goal is to predict the Sales (cont. variable) by using values from TV, Radio and Newspaper advertising
x = data.loc[:,['TV','Radio','Newspaper']].values
y = data['Sales'].values

In [126]:
x

array([[230.1,  37.8,  69.2],
       [ 44.5,  39.3,  45.1],
       [ 17.2,  45.9,  69.3],
       [151.5,  41.3,  58.5],
       [180.8,  10.8,  58.4],
       [  8.7,  48.9,  75. ],
       [ 57.5,  32.8,  23.5],
       [120.2,  19.6,  11.6],
       [  8.6,   2.1,   1. ],
       [199.8,   2.6,  21.2],
       [ 66.1,   5.8,  24.2],
       [214.7,  24. ,   4. ],
       [ 23.8,  35.1,  65.9],
       [ 97.5,   7.6,   7.2],
       [204.1,  32.9,  46. ],
       [195.4,  47.7,  52.9],
       [ 67.8,  36.6, 114. ],
       [281.4,  39.6,  55.8],
       [ 69.2,  20.5,  18.3],
       [147.3,  23.9,  19.1],
       [218.4,  27.7,  53.4],
       [237.4,   5.1,  23.5],
       [ 13.2,  15.9,  49.6],
       [228.3,  16.9,  26.2],
       [ 62.3,  12.6,  18.3],
       [262.9,   3.5,  19.5],
       [142.9,  29.3,  12.6],
       [240.1,  16.7,  22.9],
       [248.8,  27.1,  22.9],
       [ 70.6,  16. ,  40.8],
       [292.9,  28.3,  43.2],
       [112.9,  17.4,  38.6],
       [ 97.2,   1.5,  30. ],
       [26

$$\large Sales \approx w_1*\text{TV} + w_2*\text{Radio} + w_3*{Newspaper} + w_0*1 $$

In [17]:
# first we want to center the data, and then we want to find the optimal weights for a linear model, i.e. minimize the sum of squared errors
# we may need a scaler
def zscore(x):
    return (x-np.mean(x,axis=0))/np.std(x,axis=0)

In [18]:
xscaled = zscore(x)
ycentered = y-np.mean(y) # this is only to show and tell, but it is not what we prefer in practice

In [5]:
# we look for the optimal weights w1, w2 and w3 such that the predictions made with the corresponding linear combination of the columns is minimizing SSE
# define the loss function
# if we use ycentered we do not need an intercept term 
def L(w):
    prediction = xscaled@w
    errors = ycentered - prediction
    return sum((errors**2))/len(errors) # this is MSE

In [12]:
def gradient(w):
    prediction = xscaled@w
    errors = ycentered - prediction
    return (-1/len(errors))*errors@xscaled # here we applied the Chain Rule to differentiate; also the linear algebra behind matrix-vector products

In [28]:
w = [1,2,3]
L(w)

np.float64(18.62758321149871)

### We want to improve w

So, we apply an update in the direction of the negative gradient

In [29]:
w_new = w - 0.1*gradient(w)

In [30]:
L(w_new)

np.float64(15.876582967901031)

In [35]:
# we want the "optimal" value for the weights
maxiter = 2000
learning_rate = 0.01
# we want some stopping criteria, for example if the weights are no longer significantly updated
# we measure the difference between w_new and w_old vs epsilon -> tolerance for deciding convergence.
tol = 1e-6
w_old = [1,2,3]

In [36]:
# how to implement the gradient descent
for it in range(maxiter):
    w_new = w_old - learning_rate*gradient(w_old)

    if max(abs(w_new-w_old))<tol:
        print("The algorithm has converged!")
        break

    w_old = w_new
    if (it+1)%100==0:
        print('The MSE is : '+str(L(w_old)))

The MSE is : 5.529280549953874
The MSE is : 3.3339875890418336
The MSE is : 2.907894189771724
The MSE is : 2.814370267921421
The MSE is : 2.791904755916669
The MSE is : 2.7861861325459163
The MSE is : 2.7846804566616536
The MSE is : 2.784276631451753
The MSE is : 2.7841672639684334
The MSE is : 2.7841374943577
The MSE is : 2.7841293701756977
The MSE is : 2.7841271501531195
The MSE is : 2.7841265431019444
The MSE is : 2.784126377051365
The algorithm has converged!


In [37]:
w_old # these are the found optimal weights

array([ 3.91925106,  2.79190903, -0.02238423])

In [40]:
model = LinearRegression(fit_intercept=False)
model.fit(xscaled, ycentered)
model.coef_

array([ 3.91925365,  2.79206274, -0.02253861])

In [42]:
np.mean((ycentered - model.predict(xscaled))**2)

np.float64(2.784126314510936)

In [43]:
L(w_old)

np.float64(2.7841263451716043)

In [19]:
# let's repeat the gradient descent by including a bias term and by NOT centereing the values of y
x_aug = np.column_stack([np.ones(len(xscaled)),xscaled])

In [47]:
pd.DataFrame(x_aug)

0         1         2         3
0    1.0  0.969852  0.981522  1.778945
1    1.0 -1.197376  1.082808  0.669579
2    1.0 -1.516155  1.528463  1.783549
3    1.0  0.052050  1.217855  1.286405
4    1.0  0.394182 -0.841614  1.281802
..   ...       ...       ...       ...
195  1.0 -1.270941 -1.321031 -0.771217
196  1.0 -0.617035 -1.240003 -1.033598
197  1.0  0.349810 -0.942899 -1.111852
198  1.0  1.594565  1.265121  1.640850
199  1.0  0.993206 -0.990165 -1.005979

[200 rows x 4 columns]

In [109]:
# we want the "optimal" value for the weights
maxiter = 2000
learning_rate = 0.01
# we want some stopping criteria, for example if the weights are no longer significantly updated
# we measure the difference between w_new and w_old vs epsilon -> tolerance for deciding convergence.
tol = 1e-6
w_old = [0,1,2,3]

In [107]:
def L(w):
    prediction = x_aug@w
    errors = y - prediction
    return sum((errors**2))/len(errors) # this is MSE

In [106]:
def gradient(w):
    prediction = x_aug@w
    errors = y - prediction
    return (-2/len(errors))*errors@x_aug

In [110]:
# how to implement the gradient descent
for it in range(maxiter):
    w_new = w_old - learning_rate*gradient(w_old)

    if max(abs(w_new-w_old))<tol:
        print("The algorithm has converged!")
        break

    w_old = w_new
    if (it+1)%100==0:
        print('The MSE is : '+str(L(w_old)))
w_trained = w_old

The MSE is : 6.785431261566528
The MSE is : 2.8746070104389174
The MSE is : 2.787202061544548
The MSE is : 2.784290392040183
The MSE is : 2.7841373612765046
The MSE is : 2.7841271146194715
The MSE is : 2.784126373560019
The algorithm has converged!


In [113]:
# to use the weights for prediction
def predict(w,x):
    return x@w

In [117]:
def score(y,predicted):
    return 1 - sum((y-predicted)**2)/sum((y-np.mean(y))**2)

In [118]:
# so the R2-score of our predicted values is
score(y,predict(w_old,x_aug))

np.float64(0.897210637899863)

In [ ]:
# How to make a 5-Fold CV
permuted_ind = np.random.permutation(range(len(x)))
folds = np.array_split(permuted_ind,5)

for fold in folds:
    testind = fold
    trainind = np.delete(range(len(x)),fold)
    xtrain = x_aug[trainind]
    xtest = x_aug[testind]
    # here you update the weights in the train data and then you can predict on test    

In [57]:
gradient([0,1,3,4])

array([-14.0225    ,  -2.67998863,   1.47233287,   3.93080046])

In [58]:
[0,1,3,4] - 0.1*gradient([0,1,3,4])

array([1.40225   , 1.26799886, 2.85276671, 3.60691995])

## Ridge Regression

Main idea: we want to minimize the MSE with a constraint on the weights, such as sum of squared weight is less than something -> this is equivalent to the following objective 

$$\text{L-ridge}(w):= MSE + \alpha*\sum_{j=1}^{p} w_j^2$$

Critical thinking question: what is the gradient of the Ridge objective function?

$$\text{gradient-ridge}(w):= \nabla MSE + 2*\alpha*w$$

In [119]:
def L_ridge(w):
    prediction = x_aug@w 
    errors = y - prediction
    return sum((errors**2))/len(errors) + alpha*sum(w**2)

In [120]:
def gradient_ridge(w):
    prediction = x_aug@w
    errors = y - prediction
    return (-2/len(errors))*errors@x_aug + 2*alpha*np.array(w)

In [68]:
gradient([0,1,3,5]) + 2*0.01*np.array([0,1,3,5])

array([-14.0225    ,  -2.60334075,   1.88643662,   5.03080046])

In [121]:
# apply GD for training the Ridge coefficients
# we want the "optimal" value for the weights
maxiter = 2000
learning_rate = 0.01
# we want some stopping criteria, for example if the weights are no longer significantly updated
# we measure the difference between w_new and w_old vs epsilon -> tolerance for deciding convergence.
tol = 1e-6
w_old = [0,1,2,3]
alpha = 0.01

In [122]:
# how to implement the gradient descent
for it in range(maxiter):
    w_new = w_old - learning_rate*gradient_ridge(w_old)

    if max(abs(w_new-w_old))<tol:
        print("The algorithm has converged!")
        break

    w_old = w_new
    if (it+1)%100==0:
        print('The MSE is : '+str(L(w_old)))

The MSE is : 7.1249749225194785
The MSE is : 2.963671733638895
The MSE is : 2.8193454219648384
The MSE is : 2.807547251958973
The MSE is : 2.8060100860177983
The MSE is : 2.8057644119668
The MSE is : 2.805718529521846
The algorithm has converged!


## The Mini_Batch Stochastic Gradient Descent

Not getting stuck in a sub-optimal local minima!

In [10]:
# step one shuffle the data (randomly permute the order of observations)
# step two split it in mini-batches (subsets of pretty much the same cardinality)
# for updating the weights we only use gradients computed on the mini-batches
####################################

# step 1:
ind = np.random.permutation(range(len(x)))
batches = np.array_split(ind,10)

In [11]:
batches

[array([139,  14,  34,  55, 107, 166, 148, 167,   9, 109, 129,  81,  94,
        132,  21,  87, 195,  88,  53, 174]),
 array([ 44,  45,  42,  11,  82, 180,  98,  86, 170, 134, 146,  24,   0,
         36,   7, 100, 171,  27, 115,  65]),
 array([198, 183, 112, 121,  64, 190, 138, 179, 122, 114,  32, 144,  63,
         69,  10,   6, 137, 176,  38, 157]),
 array([ 99,   5, 156,  73,  12, 177, 186, 196,  85, 154,  46, 101,  23,
         66,  47,  78,  30,  74,  95,  52]),
 array([163, 123,  50,  89, 191,  83, 187,  80, 155, 142, 143, 136, 188,
        130,  79, 125, 104, 105,   3,   4]),
 array([  8, 106, 192, 165, 145, 150, 119, 153,  56,  35,  19, 117, 152,
         33,  13,  77, 120, 168, 111,  75]),
 array([ 28, 128, 149, 199,  40, 161,  72, 116, 184,  84,  29,  39, 197,
         20,  70, 102, 147, 126, 124,  31]),
 array([135, 162, 133, 169, 158, 175,  96, 181,  48,  90, 118,  57, 151,
        189, 140,   2,  17, 182,  18, 103]),
 array([ 67,  76,  59,  62,  15, 159, 110,  51, 164,  25

In [127]:
def gradient(w,ind):
    prediction = x_aug[ind]@w
    errors = y[ind] - prediction
    return (-2/len(errors))*errors@x_aug[ind]

In [124]:
# we want the "optimal" value for the weights
epochs = 4000
batch_size = 12
learning_rate = 0.01
# we want some stopping criteria, for example if the weights are no longer significantly updated
# we measure the difference between w_new and w_old vs epsilon -> tolerance for deciding convergence.
tol = 1e-5
w_old = [0,1,2,3]
done = False

In [125]:
# how to implement the gradient descent
for epoch in range(epochs):
    ind = np.random.permutation(range(len(x)))
    batches = np.array_split(ind,batch_size)
    for ind in batches:
        w_new = w_old - learning_rate*gradient(w_old,ind)
    
        if max(abs(w_new-w_old))<tol:
            done = True
            break
    
        w_old = w_new
    if (epoch+1)%200==0:
        print('The MSE is : '+str(L(w_old)))
    if done:
        print("The algorithm has converged!")
        break

The MSE is : 2.7842189191833615
The MSE is : 2.78416943127234
The MSE is : 2.784258876384771
The MSE is : 2.784263639723103
The MSE is : 2.7841842989728742
The MSE is : 2.784252372954826
The MSE is : 2.784195779044922
The MSE is : 2.784133676544374
The MSE is : 2.7841602415484603
The MSE is : 2.784369877212638
The MSE is : 2.784423654631977
The MSE is : 2.7843352208502727
The MSE is : 2.7842152038733947
The MSE is : 2.7841539779405595
The MSE is : 2.784231697251457
The MSE is : 2.784231089111365
The MSE is : 2.7843871817616925
The MSE is : 2.7845327257215975
The MSE is : 2.784135369795406
The MSE is : 2.7843243943619367


### Mini-Batch Stochastic Adaptive Gradient Descent

In [128]:
# we want the "optimal" value for the weights
epochs = 5000
batch_size = 10
learning_rate = 0.05
# we want some stopping criteria, for example if the weights are no longer significantly updated
# we measure the difference between w_new and w_old vs epsilon -> tolerance for deciding convergence.
tol = 1e-6
w_old = [0,1,2,3]
done = False
eta = 1
epsilon = 1e-6
s = 0

In [129]:
# how to implement the mini-batch adaptive gradient descent
for epoch in range(epochs):
    ind = np.random.permutation(range(len(x)))
    batches = np.array_split(ind,len(x)//batch_size)
    for ind in batches:
        g = gradient(w_old,ind)
        s += sum(g**2)
        learning_rate = eta/np.sqrt(s+epsilon)
        w_new = w_old - learning_rate*g
    
        if max(abs(w_new-w_old))<tol:
            done = True
            break
    
        w_old = w_new
    if (epoch+1)%500==0:
        print('The MSE is : '+str(L(w_old)))
    if done:
        print("The algorithm has converged!")
        break

The MSE is : 2.7841754289097493
The MSE is : 2.7841290799417724
The MSE is : 2.784133302795261
The MSE is : 2.784128055301857
The MSE is : 2.784127971234982
The MSE is : 2.784130563780534
The MSE is : 2.784126765684692
The MSE is : 2.7841268391587852
The MSE is : 2.784127930945166
The MSE is : 2.7841266169300978


## Root Mean Square Propagation

In [130]:
# we want the "optimal" value for the weights
epochs = 10000
batch_size = 100
# we want some stopping criteria, for example if the weights are no longer significantly updated
# we measure the difference between w_new and w_old vs epsilon -> tolerance for deciding convergence.
tol = 1e-5
w_old = [0,1,2,3]
done = False
eta = 0.001
epsilon = 1e-8
s = 0
gamma = 0.9

In [131]:
# how to implement the mini-batch adaptive gradient descent
for epoch in range(epochs):
    ind = np.random.permutation(range(len(x)))
    batches = np.array_split(ind,len(x)//batch_size)
    for ind in batches:
        g = gradient(w_old,ind)
        s = gamma*s +(1-gamma)*sum(g**2)
        learning_rate = eta/np.sqrt(s+epsilon)
        w_new = w_old - learning_rate*g
    
        if max(abs(w_new-w_old))<tol:
            done = True
            break
    
        w_old = w_new
    if (epoch+1)%500==0:
        print('The MSE is : '+str(L(w_old)))
    if done:
        print("The algorithm has converged!")
        break

The MSE is : 187.16783046873715
The MSE is : 161.30120279966025
The MSE is : 137.39586115456973
The MSE is : 115.44982169433682
The MSE is : 95.47493297226922
The MSE is : 77.44858352379518
The MSE is : 61.383127201626266
The MSE is : 47.273506861970766
The MSE is : 35.11536246326244
The MSE is : 24.905693842639447
The MSE is : 16.65320218281837
The MSE is : 10.336073223499197
The MSE is : 5.95635919710087
The MSE is : 3.490003669498391
The MSE is : 2.7992911637772555
The MSE is : 2.7842085359018456
The MSE is : 2.784127047456139
The MSE is : 2.7841264635002965
The MSE is : 2.7841263759536723
The MSE is : 2.784126505514004


## Adaptive Momentum Gradient Descent

In [139]:
# we want the "optimal" value for the weights
epochs = 10000
batch_size = 30
# we want some stopping criteria, for example if the weights are no longer significantly updated
# we measure the difference between w_new and w_old vs epsilon -> tolerance for deciding convergence.
tol = 1e-6
w_old = [0,1,2,3]
done = False
eta = 0.001
epsilon = 1e-8
s = 0
v = 0
beta1 = 0.9
beta2 = 0.9
t = 0

In [140]:
# how to implement the mini-batch adaptive gradient descent
for epoch in range(epochs):
    ind = np.random.permutation(range(len(x)))
    batches = np.array_split(ind,len(x)//batch_size)
    for ind in batches:
        g = gradient(w_old,ind)
        v = beta1*v + (1-beta1)*g
        s = beta2*s +(1-beta2)*sum(g**2)
        v = v/(1-beta1**(t+1))
        s = s/(1-beta2**(t+1))
        learning_rate = eta/(np.sqrt(s)+epsilon)
        w_new = w_old - learning_rate*v
        
        if max(abs(w_new-w_old))<tol:
            done = True
            break
        t += 1
        w_old = w_new
    if (epoch+1)%500==0:
        print('The MSE is : '+str(L(w_old)))
    if done:
        print("The algorithm has converged!")
        break

The MSE is : 93.44796348843207
The MSE is : 46.892041949610764
The MSE is : 17.04842817145521
The MSE is : 3.8496944871446677
The MSE is : 2.784696365984472
The MSE is : 2.784126878435726
The MSE is : 2.7841282937753236
The MSE is : 2.784127202026523
The MSE is : 2.7841274494362365
The MSE is : 2.784126706362431
The MSE is : 2.7841264882160384
The MSE is : 2.784126784530232
The MSE is : 2.7841263271015246
The MSE is : 2.784127178511048
The MSE is : 2.7841266744302784
The MSE is : 2.784126422310194
The MSE is : 2.7841264642137316
The MSE is : 2.784126654006661
The MSE is : 2.784126724344282
The MSE is : 2.78412646665901
